In [1]:
import open3d as o3d
import numpy as np
import copy
import os
import sys
import time

# monkey patches visualization and provides helpers to load geometries
sys.path.append('..')
import open3d_tutorial as o3dtut
# change to True if you want to interact with the visualization windows
o3dtut.interactive = not "CI" in os.environ

[download_utils.py] /home/rey/workspace/isl/cicp/Open3D/examples/test_data/open3d_downloads/RGBD/normal_map.npy already exists, skipped.
[download_utils.py] /home/rey/workspace/isl/cicp/Open3D/examples/test_data/open3d_downloads/RGBD/raycast_vtx_004.npy already exists, skipped.
[download_utils.py] /home/rey/workspace/isl/cicp/Open3D/examples/test_data/open3d_downloads/tests/bunnyData.pts already exists, skipped.
[download_utils.py] /home/rey/workspace/isl/cicp/Open3D/examples/test_data/open3d_downloads/tests/point_cloud_sample1.pts already exists, skipped.
[download_utils.py] /home/rey/workspace/isl/cicp/Open3D/examples/test_data/open3d_downloads/tests/point_cloud_sample2.pts already exists, skipped.
[download_utils.py] /home/rey/workspace/isl/cicp/Open3D/examples/test_data/open3d_downloads/tests/cube.obj already exists, skipped.
[download_utils.py] /home/rey/workspace/isl/cicp/Open3D/examples/test_data/open3d_downloads/RGBD/vertex_map.npy already exists, skipped.


## Helper visualization function
The function below visualizes a target point cloud and a source point cloud transformed with an alignment transformation. The target point cloud and the source point cloud are painted with cyan and yellow colors respectively. The more and tighter the two point clouds overlap with each other, the better the alignment result.

In [2]:
def draw_registration_result(source, target, transformation):
    source_temp = source.clone()
    target_temp = target.clone()
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation.to(o3d.core.Dtype.Float32))
    o3d.visualization.draw([source_temp, target_temp])

# ICP registration
This tutorial demonstrates the ICP (Iterative Closest Point) registration algorithm. It has been a mainstay of geometric registration in both research and industry for many years. The input are two point clouds and an initial transformation that roughly aligns the source point cloud to the target point cloud. The output is a refined transformation that tightly aligns the two point clouds. A helper function `draw_registration_result` visualizes the alignment during the registration process. In this tutorial, we show two ICP variants, the point-to-point ICP and the point-to-plane ICP [\[Rusinkiewicz2001\]](../reference.html#rusinkiewicz2001).

## Understanding ICP Algorithm

In general, the ICP algorithm iterates over two steps:

1. Find correspondence set $\mathcal{K}=\{(\mathbf{p}, \mathbf{q})\}$ from target point cloud $\mathbf{P}$, and source point cloud $\mathbf{Q}$ transformed with current transformation matrix $\mathbf{T}$.
2. Update the transformation $\mathbf{T}$ by minimizing an objective function $E(\mathbf{T})$ defined over the correspondence set $\mathcal{K}$.

Different variants of ICP use different objective functions $E(\mathbf{T})$ [\[BeslAndMcKay1992\]](../reference.html#beslandmckay1992) [\[ChenAndMedioni1992\]](../reference.html#chenandmedioni1992) [\[Park2017\]](../reference.html#park2017).

## Understanding ICP API

## Point-to-point ICP

We first show a point-to-point ICP algorithm [\[BeslAndMcKay1992\]](../reference.html#beslandmckay1992) using the objective

\begin{equation}
E(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\|\mathbf{p} - \mathbf{T}\mathbf{q}\|^{2}
\end{equation}

The class `TransformationEstimationPointToPoint` provides functions to compute the residuals and Jacobian matrices of the point-to-point ICP objective. The function `registration_icp` and `registration_multi_scale_icp` takes it as a parameter and runs point-to-point ICP to obtain the results.

---

## Point-to-plane ICP
The point-to-plane ICP algorithm [\[ChenAndMedioni1992\]](../reference.html#chenandmedioni1992) uses a different objective function

\begin{equation}
E(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\big((\mathbf{p} - \mathbf{T}\mathbf{q})\cdot\mathbf{n}_{\mathbf{p}}\big)^{2},
\end{equation}

where $\mathbf{n}_{\mathbf{p}}$ is the normal of point $\mathbf{p}$. [\[Rusinkiewicz2001\]](../reference.html#rusinkiewicz2001) has shown that the point-to-plane ICP algorithm has a faster convergence speed than the point-to-point ICP algorithm.

The class `TransformationEstimationPointToPlane` provides functions to compute the residuals and Jacobian matrices of the point-to-plane ICP objective. The function `registration_icp` and `registration_multi_scale_icp` takes it as a parameter and runs point-to-plane ICP to obtain the results.

---

## Colored point cloud registration

Following [\[Park2017\]](../reference.html#park2017), it runs ICP iterations with a joint optimization objective

\begin{equation}
E(\mathbf{T}) = (1-\delta)E_{C}(\mathbf{T}) + \delta E_{G}(\mathbf{T})
\end{equation}

where $\mathbf{T}$ is the transformation matrix to be estimated. $E_{C}$ and $E_{G}$ are the photometric and geometric terms, respectively. $\delta\in[0,1]$ is a weight parameter that has been determined empirically.

The geometric term $E_{G}$ is the same as the [Point-to-plane ICP](../pipelines/icp_registration.ipynb#Point-to-plane-ICP) objective

\begin{equation}
E_{G}(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\big((\mathbf{p} - \mathbf{T}\mathbf{q})\cdot\mathbf{n}_{\mathbf{p}}\big)^{2},
\end{equation}

where $\mathcal{K}$ is the correspondence set in the current iteration. $\mathbf{n}_{\mathbf{p}}$ is the normal of point $\mathbf{p}$.

The color term $E_{C}$ measures the difference between the color of point $\mathbf{q}$ (denoted as $C(\mathbf{q})$) and the color of its projection on the tangent plane of $\mathbf{p}$.

\begin{equation}
E_{C}(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\big(C_{\mathbf{p}}(\mathbf{f}(\mathbf{T}\mathbf{q})) - C(\mathbf{q})\big)^{2},
\end{equation}

where $C_{\mathbf{p}}(\cdot)$ is a precomputed function continuously defined on the tangent plane of $\mathbf{p}$. Function$\mathbf{f}(\cdot)$ projects a 3D point to the tangent plane. For more details, refer to [\[Park2017\]](../reference.html#park2017).

To further improve efficiency, [\[Park2017\]](../reference.html#park2017) proposes a multi-scale registration scheme. 

The class `TransformationEstimationForColoredICP` provides functions to compute the residuals and Jacobian matrices of the joint optimization objective. The function `registration_icp` and `registration_multi_scale_icp` takes it as a parameter and runs colored ICP to obtain the results.

---

## Input
The code below reads a source point cloud and a target point cloud from two files. A rough transformation is given.

<div class="alert alert-info">
    
**Note:** 

The initial alignment is usually obtained by a global registration algorithm. See [Global registration](../pipelines/global_registration.rst) for examples.

</div>

In [3]:
source = o3d.t.io.read_point_cloud("../../test_data/ColoredICP/frag_115.ply")
target = o3d.t.io.read_point_cloud("../../test_data/ColoredICP/frag_116.ply")

# For Colored-ICP `colors` attribute must be of same dtype as `positions` and `normals` attribute.
source.point["colors"] = source.point["colors"].to(
    o3d.core.Dtype.Float32) / 255.0
target.point["colors"] = target.point["colors"].to(
    o3d.core.Dtype.Float32) / 255.0

trans_init = o3d.core.Tensor.eye(4, o3d.core.Dtype.Float32)

draw_registration_result(source, target, trans_init)

../src/intel/isl/isl.c:2018: FINISHME: ../src/intel/isl/isl.c:isl_surf_supports_ccs: CCS for 3D textures is disabled, but a workaround is available.


---

#### Python API » open3d.t » open3d.t.pipelines » open3d.t.pipelines.registration » open3d.t.pipelines.registration.registration_multi_scale_icp

## Parameters 

### Input PointClouds between which the `Transformation` is to be estimated. [open3d.t.PointCloud]
- Source Tensor PointCloud. [Float32 or Float64 dtypes are supported].
- Target Tensor PointCloud. [Float32 or Float64 dtypes are supported].

### Max correspondence Distances
- This is the radius of distance from each point in source point-cloud in which the neighbour search will try to find an corresponding point in the target point-cloud.
- It is a `double` for `ICP`, and `utility.DoubleVector` for `Multi-Scale-ICP`.
- One may typically keep this parameter between 1.0x - 3.0x `voxel-size` for each scale.
- This parameter is most important for performance tuning, as higher radius will take larget time (as the neighbour search will be performed over a larger radius).

### Inital Transform from Source to Target [open3d.core.Tensor]
- Initial estimate for transfromation from source to target.
- Transformation matrix Tensor of shape [4, 4] of type `Float64` on `CPU:0` device
- The initial alignment is usually obtained by a global registration algorithm. See [Global registration](../pipelines/global_registration.rst) for examples.

### Estimation Method 
- This sets the ICP method to compute the transformation between two point-clouds given the correspondences.

Options:

- **o3d.t.pipelines.registration.TransformationEstimationPointToPoint()**
    - Point to Point ICP.
- **o3d.t.pipelines.registration.TransformationEstimationPointToPlane(robust_kernel)**
    - Point to Plane ICP.
    - Requires `target point-cloud` to have `normals` attribute (of same dtype as `position` attribute).
- **o3d.t.pipelines.registration.TransformationEstimationForColoredICP(robust_kernel, lambda)**
    - Colored ICP.
    - Requires `target` point-cloud to have `normals` attribute (of same dtype as `position` attribute).
    - Requires `source` and `target` point-clouds to have `colors` attribute (of same dtype as `position` attribute).
- **o3d.t.pipelines.registration.TransformationEstimationForGeneralizedICP(robust_kernel, epsilon)** [To be added].
    - Generalized ICP.


#### Estimation Method supports `Robust Kernels`. 
- Robust kernels are used for outlier rejection.

`robust_kernel = o3d.t.pipelines.registration.robust_kernel.RobustKernel(method, scale, shape)`
Method options
- robust_kernel.RobustKernelMethod.L2Loss
- robust_kernel.RobustKernelMethod.L1Loss
- robust_kernel.RobustKernelMethod.HuberLoss
- robust_kernel.RobustKernelMethod.CauchyLoss
- robust_kernel.RobustKernelMethod.GMLoss
- robust_kernel.RobustKernelMethod.TurkeyLoss
- robust_kernel.RobustKernelMethod.GeneralizedLoss


### ICP Convergence Criteria [relative rmse, relative fitness, max iterations]
- This sets the condition for termination or when the scale iterations can be considered to be converged. 
- If the relative (of change in value from last iteration) rmse and fitness is equal or less than the specified value, the iterations for that scale will be considered as converged / completed.
- For `Multi-Scale ICP` it is a `list` of `ICPConvergenceCriteria`, for each scale of ICP, to provide more fine control over performance.
- One may keep the initial values of `relative_fitness` and `relative_rmse` low as we just want to get an estimate transformation, and high for later iterations to fine-tune.
- Iterations on higher-resolution is more costly (takes more time), so we want to do less iterations on higher resolution.

### Voxel Sizes [open3d.utility.DoubleVector]
- It is the voxel size (analogous to resolution in case of image), for each scale of mult-scale ICP.
- We want to perform initial iterations on a coarse point-cloud (low-resolution or small voxel size) (as it is more time-efficient, and avoids local-minima), and then move to dense point-cloud (high-resolution or small voxel size. Therefore the voxel sizes must be strictly decreasing order.

---

## ICP Example

In [4]:
if o3d.__DEVICE_API__ == 'cuda':
    import open3d.cuda.pybind.t.pipelines.registration as treg
else:
    import open3d.cpu.pybind.t.pipelines.registration as treg

In [5]:
criteria = treg.ICPConvergenceCriteria(relative_fitness=0.000001,
                                       relative_rmse=0.000001,
                                       max_iteration=50)

max_correspondence_distances = 0.03

init_source_to_target = trans_init

estimation = treg.TransformationEstimationForColoredICP(
    treg.robust_kernel.RobustKernel(
        treg.robust_kernel.RobustKernelMethod.L2Loss))

In [6]:
s = time.time()
source_icp = source.voxel_down_sample(0.0125)
target_icp = target.voxel_down_sample(0.0125)

registration_result = treg.registration_icp(source_icp, target_icp,
                                            max_correspondence_distances,
                                            init_source_to_target, estimation,
                                            criteria)

icp_time = time.time() - s
print("Time taken by ICP: ", icp_time)
print("Inlier Fitness: ", registration_result.fitness)
print("Inlier RMSE: ", registration_result.inlier_rmse)

draw_registration_result(source, target, registration_result.transformation)

[Open3D WARNING] Use voxel size parameter, for better performance in ColoredICP.
Time taken by ICP:  0.2549111843109131
Inlier Fitness:  0.881921214239353
Inlier RMSE:  0.007933636358403643


---

## Multi-Scale ICP Example

In [7]:
voxel_sizes = o3d.utility.DoubleVector([0.05, 0.025, 0.0125])

criteria_list = [
    treg.ICPConvergenceCriteria(relative_fitness=0.0001,
                                relative_rmse=0.0001,
                                max_iteration=20),
    treg.ICPConvergenceCriteria(0.00001, 0.00001, 15),
    treg.ICPConvergenceCriteria(0.000001, 0.000001, 10)
]

max_correspondence_distances = o3d.utility.DoubleVector([0.7, 0.07, 0.03])

init_source_to_target = trans_init

estimation = treg.TransformationEstimationForColoredICP(
    treg.robust_kernel.RobustKernel(
        treg.robust_kernel.RobustKernelMethod.L2Loss))

In [8]:
# Setting Verbosity to Debug, helps in fine-tuning the performance.
# o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)

s = time.time()

registration_result = treg.registration_multi_scale_icp(
    source, target, voxel_sizes, criteria_list, max_correspondence_distances,
    init_source_to_target, estimation)

ms_icp_time = time.time() - s
print("Time taken by Multi-Scale ICP: ", ms_icp_time)
print("Inlier Fitness: ", registration_result.fitness)
print("Inlier RMSE: ", registration_result.inlier_rmse)

draw_registration_result(source, target, registration_result.transformation)

Time taken by Multi-Scale ICP:  0.16813993453979492
Inlier Fitness:  0.8817215038194618
Inlier RMSE:  0.007867755949250715


---

## Multi-Scale ICP on CUDA device Example

In [9]:
# The algorithm runs on the same device as the source and target point-cloud.
source = source.cuda(0)
target = target.cuda(0)

s = time.time()

registration_result = treg.registration_multi_scale_icp(
    source, target, voxel_sizes, criteria_list, max_correspondence_distances,
    init_source_to_target, estimation)

ms_icp_time = time.time() - s
print("Time taken by Multi-Scale ICP: ", ms_icp_time)
print("Inlier Fitness: ", registration_result.fitness)
print("Inlier RMSE: ", registration_result.inlier_rmse)

draw_registration_result(source.cpu(), target.cpu(),
                         registration_result.transformation)

FEngine (64 bits) created at 0x7f668c75b010 (threading is enabled)
FEngine resolved backend: OpenGL
Time taken by Multi-Scale ICP:  0.040312767028808594
Inlier Fitness:  0.8821708522642169
Inlier RMSE:  0.007836136137721585


---

### Information Matrix

In [10]:
information_matrix = treg.get_information_matrix_from_point_clouds(
    source, target, max_correspondence_distances[2],
    registration_result.transformation)

print(information_matrix)

[[329337. -195719. -104339. 0.0 -65782.2 117300.],
 [-195719. 251265. -136397. 65782.2 0.0 -95464.3],
 [-104339. -136397. 423152. -117300. 95464.3 0.0],
 [0.0 65782.2 -117300. 56979.0 0.0 0.0],
 [-65782.2 0.0 95464.3 0.0 56979.0 0.0],
 [117300. -95464.3 0.0 0.0 0.0 56979.0]]
Tensor[shape={6, 6}, stride={6, 1}, Float64, CPU:0, 0x55d87ed476d0]
